In [1]:
import pandas as pd
import numpy as np
import pydeck as pdk
from os import environ, getenv, path
from typing import TypedDict, Dict, Union, List
from simulator_funcs import *
import warnings

warnings.filterwarnings("ignore")

from odp.geospatial.odp_vessel_simulator.models.icct.database_functions.lookup_ship_data import fetch_ship_data
from odp.geospatial.odp_vessel_simulator.models.icct.database_functions.database import get_connection_pool, get_engine
import typing


## Color code explanation
<p style="color:#FE774C; font-size:12pt;"> Red means that values <i>have</i> to be inserted.</p> 
<p style="color:#9D59F4; font-size:12pt"> Purple means that values <i>can</i> be inserted if other that the default values are needed.</p> 


<p> </p> 
 <p style="color:#FE774C; font-size:20pt; font-weight:bold"> Insert all the mmsis for the vessels interest in here </p> 
<p> For example like the list under with two vessels (change the mmsis and add more of them if needed). </p> 

In [2]:
mmsis = [212552000, 319009700, 352297000,477630400]

<p> </p> 
 <p style="color:#FE774C; font-size:20pt; font-weight:bold"> Insert the names of the ports in the simulated paths </p> 
<p> For example like the list under where the path will go from Oslo to Kiel to Amsterdam. </p> 

In [3]:
ports = ['oslo','kiel','amsterdam']

In [4]:
df_ports, coordinates = get_lon_lat_ports(ports)

In [5]:
df_ports

,port_name,country,longitude,latitude
0,OSLO,NO,10.750000,59.916667
1,KIEL,DE,10.133333,54.316667
2,AMSTERDAM,NL,4.900000,52.366667


In [6]:
coordinates

[[10.75, 59.916667], [10.133333, 54.316667], [4.9, 52.366667]]

#### If the port does not exist, the coordinates must be added manually
This is done by uncommenting the line below and changing the index, 'longitude' and 'latitude' with the actual numbers. Lists are 0-indexed such that if the missing port is the first one, the index is 0.

In [7]:
#coordinates.insert(index,[longitude,latitude])

# Specifying properties needed for routing

In the following cells we are specifying speed and draught for the vessels as well as graphing, time resolution, how detailed the routing should be and how much other vessel data should be weighed for the routing. 

<p style="color:#9D59F4; font-size:15pt; font-weight:bold"> Make a list for the speeds to simulate with for each vessel  </p> 
<p> If the speed is set to None, the maximum speed for the vessel is chosen. </p> 

In [8]:
speeds = [None] * len(mmsis)
# Or:
# speeds = [10, 10] #Here, the speed for both vessels is set to 10 knots
speeds

[None, None, None, None]

<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> List of draught for the different vessels </p> 
<p> If they are set to None, the draught will be found in the data about the vessel particulars. </p> 

In [9]:
draughts = [None] * len(mmsis)
# Or:
# draughts = [7, 9] #here the draught for the first ship is 7m and for the second ship it is 9 m
draughts

[None, None, None, None]

<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> Choose grapfing types for the vessels </p> 
<p> This is also found from the vessel particulars, but in the incidences where it cannot be found, it needs to be specified. Possible graph types are passenger, fishing, cargo, tanker and tug. </p> 

In [10]:
graphs = [None] * len(mmsis)
# Or:
# graphs = ['cargo', 'tug']
graphs

[None, None, None, None]

### Make a dataframe of the mmsis, speeds, draughts and graphs
This is used as input in the function finding routes and emissions.

In [11]:
zipped_lists = list(zip(speeds, draughts, graphs))
df_input_properties = pd.DataFrame(zipped_lists, index = mmsis, columns = ["speeds","draughts","graphs"])


In [12]:
df_input_properties

,speeds,draughts,graphs
212552000,None,None,None
319009700,None,None,None
352297000,None,None,None
477630400,None,None,None


<p> </p>
<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> Choose the time resolution for the graphing </p> 
<p> The default value is every 60th minute. Choose a value between 5 and 240. </p> 

In [13]:
time_resolution = 60

<p> </p>
<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> Decide how fine/coarse the routing should be </p> 

 * 0 = Coarse (fast)
 * 1 = Coarse + refined
 * 2 = Fine (slow)

In [14]:
detailed_routing = 1

<p> </p>
<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> How much should popular routes be weighted in the routing? </p> 
<p> Choose a value between 0 and 0.95. Default is 0.3. </p> 

In [15]:
cost_density = 0.3

## Finding paths and emissions for each vessel

Using a premade function called emissions_multiple_vessels that takes the coordinates chosen and the df_input_properties as arguments. OBS, this might take a while. 

In [ ]:
%%time
df_emissions, df_summed_emissions = emissions_and_paths(
    coordinates,
    df_input_properties,
    time_resolution,
    detailed_routing,
    cost_density
)

Path finding for vessels with mmmsis 212552000 352297000 477630400 is done. 


### Summed emissions for all vessels compared

The vessel at the top is the one that emitts the least amount of CO2.

In [ ]:
df_summed_emissions

## Show the paths
This one is for the vessel that uses the least amount of CO2

In [ ]:
mmsi0 = df_summed_emissions.index[0]
print(f"Emissions for vessel with mmsi {mmsi0}")
plot_df(df_emissions['emissions_on_path'].loc[mmsi0])

In [ ]:
mmsi1 = df_summed_emissions.index[1]
print(f"Emissions for vessel with mmsi {mmsi1}")
plot_df(df_emissions['emissions_on_path'].loc[mmsi1])

If you are comparing more than two vessels and want to see the paths of the other ones, uncomment the line underneath. If there are more than two vessels you can also change the index to see the other (OBS, zero indexed). 

In [ ]:
#plot_df(df_emissions['emissions_on_path'].loc[df_summed_emissions.index[2]])